In [4]:
import sys
import os
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

from password.password import *

browser = webdriver.Chrome()
browser.get("https://www.rakuten-sec.co.jp/ITS/V_ACT_Login.html")

id = browser.find_element_by_id("form-login-id")
id.send_keys(ID)
password = browser.find_element_by_id("form-login-pass")
password.send_keys(PASSWORD)

time.sleep(1)

# ログインボタンをクリック
login_button = browser.find_element_by_id("login-btn")
login_button.click()
browser.find_element_by_xpath('//*[@id="gmenu_invest"]/a').click()
time.sleep(1)

try:
    browser.find_element_by_class_name('modal__button--normal').click()
except:
    pass

time.sleep(1)

browser.find_element_by_xpath('//*[@id="show_asset_total_table"]/span/table/tbody/tr/td[3]/span[2]/span[1]/a').click()
tbl_element = browser.find_element_by_id('poss-tbl-sp')

<selenium.webdriver.remote.webelement.WebElement (session="b84f2f4af536e02f009d4bfc6c1e225f", element="0.0980181938912228-1")>

In [6]:
target_list = ["ひふみプラス","AI(人工知能)活用型世界株ファンド(ディープAI)","eMAXIS Slim 先進国株式インデックス","グローバルAIファンド","グローバルAIファンド(為替ヘッジあり)","楽天日本株4.3倍ブル","ベトナム成長株インカムファンド"]
trs = tbl_element.find_elements_by_tag_name("tr")

for i in range(1,len(trs)):
    tds = trs[i].find_elements(By.TAG_NAME, "td")
    for j in tds:
        try:
            if str(j.text) in target_list:
                print(j.text)
                print(j.find_element_by_css_selector("div.mbody").find_element_by_css_selector("a").get_attribute("href"))
        except:
            pass

ひふみプラス
https://member.rakuten-sec.co.jp/app/info_fu_invest_detail.do;BV_SessionID=3404C873382D3532E1A194C980E7E778.42ad2c3e?eventType=init&SS_URL=detail/index.html&dscrCd=1410806&gmn=S&smn=01&lmn=01&fmn=01
AI(人工知能)活用型世界株ファンド(ディープAI)
https://member.rakuten-sec.co.jp/app/info_fu_invest_detail.do;BV_SessionID=3404C873382D3532E1A194C980E7E778.42ad2c3e?eventType=init&SS_URL=detail/index.html&dscrCd=2312820&gmn=S&smn=01&lmn=01&fmn=01
楽天日本株4.3倍ブル
https://member.rakuten-sec.co.jp/app/info_fu_invest_detail.do;BV_SessionID=3404C873382D3532E1A194C980E7E778.42ad2c3e?eventType=init&SS_URL=detail/index.html&dscrCd=4700831&gmn=S&smn=01&lmn=01&fmn=01
楽天日本株4.3倍ブル
https://member.rakuten-sec.co.jp/app/info_fu_invest_detail.do;BV_SessionID=3404C873382D3532E1A194C980E7E778.42ad2c3e?eventType=init&SS_URL=detail/index.html&dscrCd=4710831&gmn=S&smn=01&lmn=01&fmn=01
eMAXIS Slim 先進国株式インデックス
https://member.rakuten-sec.co.jp/app/info_fu_invest_detail.do;BV_SessionID=3404C873382D3532E1A194C980E7E778.42ad2c3e?event

In [ ]:
trs

In [ ]:



class SearchMercariData(object):
    def __init__(self,input_data):
        self.input_data = input_data
        self.num_page = 1

    def search_from_sold_list(self):
        df_url = self.get_url(self.num_page)
        df = self.get_details(df_url)
        return df

    def get_url(self, num_page):
        browser = webdriver.Chrome()
        browser.get("https://www.mercari.com/jp/search/?keyword=")
        browser.find_element_by_xpath("/html/body/div/main/div[2]/form/div[2]/div[8]/div/div[2]/label").click()
        browser.find_element_by_xpath("/html/body/div/main/div[2]/form/div[2]/div[10]/button").click()
        df = pd.DataFrame()
        page=0
        for i in range(num_page):
            if len(browser.find_elements_by_css_selector("li.pager-next .pager-cell:nth-child(1) a")) > 0:
                print("######################page: {} ########################".format(page))
                print("Starting to get posts...")
                posts = browser.find_elements_by_css_selector(".items-box")
                for post in posts:
                    url = post.find_element_by_css_selector("a").get_attribute("href")
                    box_photo = post.find_element_by_css_selector(".items-box-photo")
                    pic = box_photo.find_element_by_css_selector("img").get_attribute("data-src")
                    se = pd.Series([url,pic],index=['url','pic'])
                    df = df.append(se, ignore_index=True)
                page+=1

                btn = browser.find_element_by_css_selector("li.pager-next .pager-cell:nth-child(1) a").get_attribute("href")
                print("next url:{}".format(btn))

                df = df.drop_duplicates()
                browser.get(btn)
                print("Moving to next page......")
            else:
                print("no pager exist anymore")
        return df


    def get_details(self,df_sold_out):
        df_details_list = pd.DataFrame()
        browser = webdriver.Chrome()
        for i in range(df_sold_out.shape[0]):
            url  = df_sold_out["url"].values[i]
            pic  = df_sold_out["pic"].values[i]
            browser.get(url)
            try:
                item_name =browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/h2").text
                try:
                    sold_badge =browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/div[1]/div/div[2]/div").text
                except:
                    sold_badge = "NOT_SOLD"
                main_category =browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/div[1]/table/tbody/tr[2]/td/a[1]/div").text
                sub1_category =browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/div[1]/table/tbody/tr[2]/td/a[2]/div").text
                sub2_category = browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/div[1]/table/tbody/tr[2]/td/a[3]/div").text
                brand_name = browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/div[1]/table/tbody/tr[3]/td").text

                box_details = browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/div[1]/table").text
                list_item_condition = ["新品、未使用","未使用に近い","目立った傷や汚れなし","やや傷や汚れあり","傷や汚れあり"]
                box_details = box_details.replace("\n"," ").split(" ")

                for word in list_item_condition:
                    try:
                        item_condition = box_details[box_details.index(word)]
                        break
                    except:
                        next
                shipping_fee =  browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/div[2]/span[3]").text
                item_description= browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/div[3]").text
                price = browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/div[2]/span[1]").text

                item_description = item_description.replace("\n"," ")
                price = price.replace("¥ ","")
                se = pd.Series([item_name, sold_badge, main_category,sub1_category,sub2_category,brand_name,item_condition,shipping_fee,item_description,price,url,pic],
                                   ["item_name", "sold_badge","main_category","sub1_category","sub2_category","brand_name","item_condition","shipping_fee","item_description","price","url","pic"])
                df_details_list = df_details_list.append(se, ignore_index=True)
                print("{} details of {} was added".format(i,url) )
            except:
                print("{} are not found".format(url))
        return df_details_list